# 1.初始化环境
只考虑Colab环境, 不考虑本地NoteBook。

## 1.1 显卡驱动检查
查看CUDA版本以供PyTorch安装使用。

如果执行失败, 检查运行时中是否正确选择GPU。

## 1.2 PyTorch安装
[PyTorch官网](https://pytorch.org/)

暂时配置: Stable、Linux、Pip、Python、CUDA 11.3

使用官网给出的命令安装

## 1.3 挂载Google Drive
1. 不更改工作目录, 网络I/O通讯太慢。
2. 训练集直接拷贝到Colab本地。
3. **断点路径一定在Google Drive而不是Colab本地。**

以上问题在Config类中配置。

In [ ]:
# 1.1显卡驱动检查
!nvidia-smi

Tue Sep  6 08:34:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 1.2 PyTorch安装
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113


In [ ]:
# 1.3 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# 2.代码部分

In [ ]:
import time
import torch
from torch.nn import Module, LSTM, Linear, BatchNorm1d
from torch.utils.data import DataLoader, TensorDataset  # 常用工具区关于data区
import numpy as np  # 引入numpy
from sklearn.model_selection import train_test_split

import os

In [ ]:
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive"
PROJECT_NAME = "fruit"
PROJECT_PATH = os.path.join(GOOGLE_DRIVE_PATH, PROJECT_NAME)


In [ ]:
class Lstm_model(Module):
    def __init__(self):  # 初始化
        super(Lstm_model, self).__init__()
        self.lstm1 = LSTM(input_size=1000, hidden_size=512,
                          num_layers=1, batch_first=True)
        self.lstm2 = LSTM(input_size=1000, hidden_size=512,
                          num_layers=1, batch_first=True)
        self.lstm3 = LSTM(input_size=1000, hidden_size=512,
                          num_layers=1, batch_first=True)
        self.linear1 = Linear(in_features=512, out_features=1)
        self.linear2 = Linear(in_features=512, out_features=1)
        self.linear3 = Linear(in_features=512, out_features=1)
        self.linear = Linear(in_features=18, out_features=1)
        self.active = torch.nn.LeakyReLU()
        self.bn = BatchNorm1d(num_features=18)

    def forward(self, x, hidden=None):
        x = x.reshape(1, -1, 1000)
        # print(x.shape)
        x01 = x[0, 0:6, :]
        x02 = x[0, 6:12, :]
        x03 = x[0, 12:18, :]
        x01 = x01.reshape(1, -1, 1000)
        x02 = x02.reshape(1, -1, 1000)
        x03 = x03.reshape(1, -1, 1000)
        x1, _ = self.lstm1(x01, hidden)
        x2, _ = self.lstm2(x02, hidden)
        x3, _ = self.lstm3(x03, hidden)
        # print(x1.shape)
        # print(x2.shape)
        # print(x3.shape)
        x1 = self.linear1(x1)
        x1 = self.active(x1)
        x2 = self.linear2(x2)
        x2 = self.active(x2)
        x3 = self.linear3(x3)
        x3 = self.active(x3)
        # print(x1.shape)
        # print(x2.shape)
        # print(x3.shape)
        x = torch.cat((x1, x2, x3), dim=1).reshape(-1)
        # print(x.shape)
        x = self.linear(x)
        # print(x.shape)
        return x

In [ ]:
device = torch.device('cuda:0')
model = Lstm_model().to(device)
# dataset = train_data(r'E:\peach\data')
# print(dataset.shape)
dataset = train_data(r'E:\peach\data').reshape(-1, 18, 1000)  # reshape改为一个三维数组（-1是模糊控制，行未知，18列，1000页）
print(dataset.shape)
labelset = np.array(get_label(r'E:\peach\油桃硬度.xlsx'))
print(labelset.shape)
train_X, valid_X, train_Y, valid_Y = train_test_split(dataset, labelset,
                                                      test_size=0.2, random_state=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)  # 学习率
criterion = torch.nn.MSELoss()
train_X, train_Y = torch.from_numpy(train_X).float(), torch.from_numpy(train_Y).float()
train_loader = DataLoader(TensorDataset(train_X, train_Y), batch_size=1, shuffle=True)
valid_X, valid_Y = torch.from_numpy(valid_X).float(), torch.from_numpy(valid_Y).float()
valid_loader = DataLoader(TensorDataset(valid_X, valid_Y), batch_size=1, shuffle=True)
